### Workaround the problems of unittest.mock using `spec` - Specifications

* When configuring a Mock, we can pass an object specification to the `spec` parameter. The `spec` parameter accepts a list of names or another object and defines the mock's interface. If you attempt to access an attribute that does not belong to the specification, Mock will raise an `AttributeError`

In [12]:
from unittest.mock import Mock
calendar = Mock(spec=['is_weekday', 'get_holidays'])

print(calendar.is_weekday())

print(calendar.create_event())

<Mock name='mock.is_weekday()' id='4543972016'>


AttributeError: Mock object has no attribute 'create_event'

* In the above snipped, we specified that calendar has methods called `is_weekday()` and `get_holidays()`. When we access `is_weekday()`, it returns a `Mock`. When we access `create_event()`, a method that does not match the specification, `Mock` raises an `AttributeError`.

In [3]:
# Specifications work the same way if you configure the Mcok with an object
## Restart the kernel to remove previous imports before running this block
import my_calendar
from unittest.mock import Mock

calendar = Mock(spec=my_calendar)
print(calendar.is_weekday()) # Works
print(calendar.create_event()) # Raises an AttributeError

# is_weekday() is available to calendar because we configured `calendar` to match
# `my_calendar` module's interface

<Mock name='mock.is_weekday()' id='4584037904'>


AttributeError: Mock object has no attribute 'create_event'

### `create_autospec`

* `unittest.mock` provides convenient methods of automatically specifying a `Mock` instance's interface
* One way to implement automatic specification is `create_autospec`

In [ ]:
# Restart kernel to remove previous imports
import my_calendar
from unittest.mock import create_autospec

calendar = create_autospec(my_calendar)
print(calendar.is_weekday()) # Works
print(calendar.create_event()) # Raises AttributeErro

In [1]:
# Restart kernel to remove previous imports

# if you are using `patch()`, you can send an argument to the `autospec` 
# parameter to achieve the same result
import my_calendar
from unittest.mock import patch

with patch('__main__.my_calendar', autospec=True) as calendar:
    print(calendar.is_weekday()) # Works
    print(calendar.create_event()) # Raises AttributeError

<MagicMock name='my_calendar.is_weekday()' id='4515708880'>


AttributeError: Mock object has no attribute 'create_event'

In [6]:
# Mocking and using exactly "my_calendar.is_weekday()"

import my_calendar
from unittest.mock import patch

with patch('my_calendar.is_weekday'):
    print(my_calendar.is_weekday())
    
# We see the mocked object hence it confirms that mocking worked in desired manner

<MagicMock name='is_weekday()' id='4543995128'>


In [9]:
# Lets change the above example slightly and import the function directly
from my_calendar import is_weekday
from unittest.mock import patch

with patch("my_calendar.is_weekday"):
    print(is_weekday())

# Since I am running this on a weekend (Saturday), I am seeing the output
# as False instead of getting the mocked object

# Even though the target location passed to patch() did not change, the result
# of calling is_weekday() is different. Difference is due to the change in
# how we imported the function

# from my_calendar import is_weekday binds the real function to the local 
# scope.
# So, even though you patch() the function later, you ignore the mock because
# you already have a local reference to the un-mocked function.

# Good rule of thumb - patch() the object where it is looked up
# Ref: https://docs.python.org/3/library/unittest.mock.html#where-to-patch

# In frist example, mocking 'my_calendar.is_weekday()` works because we
# look up the function in the my_calendar module.

# In the second example, we have a local reference to `is_weekday()` hence since
# we use the function found in the local scope, we should mock the local function

with patch('__main__.is_weekday'):
    print(is_weekday())

False
<MagicMock name='is_weekday()' id='4543964328'>


In [14]:
from unittest.mock import Mock
mocked = Mock('some_func')
dir(mocked)
# print(mocked().method_calls)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mod__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmod__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'assert_any_call',
 'assert_called',
 'assert_called_once',
 'assert_called_once_with',
 'assert_called_with',
 'assert_has_calls',
 'assert_not_called',
 'attach_mock',
 'call_args',
 'call_args_list',
 'call_count',
 'called',
 'capitalize',
 'casefold',
 'center',
 'configure_mock',
 'count',
 'encode',
 'endswith',
 'expandtabs',
 'find',
 'format',
 'format_map',
 'index',
 'isalnum',
 'isalpha',
 'isascii',
 'isdecimal',
 'isdigit',
 'isidentifier',
 'islower',
 'isnumeric',
 'isprintable',
 'isspace',
 'istitle',
 'isuppe

In [4]:
mock

NameError: name 'mock' is not defined

In [4]:
mock = Mock(return_value=True)
mock()

True

In [5]:
# .side_effect and .return_value can be set on the Mock instance
# other attribs like .name can only be set through .__init__() or .configure_mock()

mock = Mock(name='Real Python Mock')

In [6]:
mock.name

<Mock name='Real Python Mock.name' id='4426887408'>

In [7]:
mock = Mock()
mock.name = 'Real Python Mock'
mock.name

'Real Python Mock'

In [8]:
# Configure an existing mock:
mock = Mock()
mock.configure_mock(return_value=True)
mock()

True

In [11]:
# By unpacking a dict into either .configure_mock() or Mock.__init__(), we
# can configure Python mock object's attribs

# Verbose old Mock
response_mock = Mock()
response_mock.json.return_value = {
    '12/25': 'Christmas',
    '7/4': 'Independence Day'
}

# Shiny, new .configure_mock()
holidays = {'12/25': 'Christmas', '7/4': 'Independence Day'}
response_mock = Mock(**{'json.return_value': holidays})

response_mock.json.return_value

{'12/25': 'Christmas', '7/4': 'Independence Day'}

### Using mock to control behaviour
### Substitute mocks for real objects in other modules